In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Declaracion de variables

# Data actual clasificada por categoria
df_actual = pd.read_csv('InformationByTokenClass.csv')

# Directorio de los datasets
DATA_PATH = ''

# Años a extraer
years = ['2019', '2020', '2021', '2022', '2023','2024']

# Lista de lista de los datasets cargados por año
df_per_year = [] # 0: 2019 - 1: 2020 - 2: 2021 - 3: 2022 - 4: 2023

# Funcion para leer todos los datasets en un año, y poner esa lista en la lista de los datasets por año
def load_data(year):
  folder_path = DATA_PATH + year + '/'
  print("Cargando data desde " + folder_path)
  csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
  dfs = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
  df_per_year.append(dfs)

# Aplicamos la funcion a cada año que queremos extraer
for year in years:
  load_data(year)


# Eliminamos las listas internas para que nos queden solo los datsets en una sola lista
df_per_year = [dataset for sublist in df_per_year for dataset in sublist]

Cargando data desde 2019/
Cargando data desde 2020/
Cargando data desde 2021/
Cargando data desde 2022/
Cargando data desde 2023/
Cargando data desde 2024/


In [29]:
# Funcion para crear un dataset con los tokens presentes en el diccionario iterando en la lista de datasets
def filter_and_combine_datasets(name,symbol, datasets):
  # Crear una lista vacía para almacenar los DataFrames filtrados
  filtered_dfs = []

  # Iterar sobre la lista de datasets
  for dataset in datasets:
      # Filtrar filas donde el nombre está en las llaves del diccionario
      filtered_df = dataset[(dataset['name'] == name) & (dataset['symbol'] == symbol)]
      # Agregar el DataFrame filtrado a la lista
      filtered_dfs.append(filtered_df)

  # Combinar todos los DataFrames filtrados en uno solo
  combined_df = pd.concat(filtered_dfs, ignore_index=True)

  # Ordenar el DataFrame combinado por la columna 'date'
  combined_df = combined_df.sort_values(by='last_updated').reset_index(drop=True)

  # Eliminar la columna 'maxSupply'
  combined_df = combined_df.drop(columns=['maxSupply'], errors='ignore')

  return combined_df

In [24]:

def print_high_null_columns(df, threshold=0.8):
    """
    Imprime las columnas que tienen más del 'threshold' de valores nulos.
    
    :param df: DataFrame a analizar
    :param threshold: Umbral para detectar columnas con alto porcentaje de nulos
    """
    # Calcular el porcentaje de valores nulos por columna
    null_percentage = df.isna().mean()

    # Encontrar las columnas que exceden el umbral
    high_null_columns = null_percentage[null_percentage > threshold]

    # Imprimir columnas con más del umbral de nulos
    if not high_null_columns.empty:
        print("Columnas con más del {:.0%} de valores nulos:".format(threshold))
        print(high_null_columns)
    else:
        print("Ninguna columna tiene más del {:.0%} de valores nulos.".format(threshold))

# Funcion para calcular cuantos valores nulos hay en cada columna agrupado por nombre de token
def count_nulls(df):
  # Contar los NaN por columna dentro de cada grupo de 'name'
  nan_counts_by_group = df.groupby('name').apply(lambda x: x.isna().sum())

  return nan_counts_by_group

def cleanData(dataset):
    df_final = dataset.replace(0, np.nan)
    print_high_null_columns(df_final, threshold=0.7)
    df_final = fill_nulls(df_final)
    return dataset

# funcion para rellenar nulos
def fill_nulls(df):
    import pandas as pd

    # Asegurarse de que las columnas de fecha estén en formato datetime
    df['last_updated'] = pd.to_datetime(df['last_updated'])
    df['dateAdded'] = pd.to_datetime(df['dateAdded'])

    # Agrupar por 'name' para iterar sobre cada token
    filled_data = []

    for name, group in df.groupby('name'):
        # Verificar tipos de datos y convertir a float si es necesario
        numeric_cols = ['circulatingSupply', 'volume24h', 'marketCap',
                        'percentChange1h', 'percentChange24h', 'percentChange7d']
        group[numeric_cols] = group[numeric_cols].apply(pd.to_numeric, errors='coerce')

        # Interpolación para las columnas especificadas
        for col in numeric_cols:
            group[col] = group[col].interpolate(method='linear', limit_direction='both')

        # Llenar los extremos si quedan nulos después de la interpolación
        group[numeric_cols] = group[numeric_cols].fillna(method='ffill').fillna(method='bfill')

        # Añadir el grupo procesado a la lista de resultados
        filled_data.append(group)

    # Combinar todos los grupos en un DataFrame final
    filled_df = pd.concat(filled_data, ignore_index=True)
    return filled_df



In [33]:
df_final = filter_and_combine_datasets('dogwifhat','WIF', df_per_year)
cleanData(df_final)

Ninguna columna tiene más del 70% de valores nulos.


,name,symbol,cmcRank,circulatingSupply,totalSupply,dateAdded,last_updated,price,volume24h,marketCap,percentChange1h,percentChange24h,percentChange7d
0,dogwifhat,WIF,370,9.989202e+08,9.989202e+08,2023-12-19,2024-01-07,0.089642,1.088928e+07,8.954552e+07,1.680609,-4.598337,-41.397028
1,dogwifhat,WIF,223,9.989202e+08,9.989202e+08,2023-12-19,2024-01-14,0.317373,1.068237e+08,3.170301e+08,-14.123444,15.119590,254.043463
2,dogwifhat,WIF,157,9.989202e+08,9.989202e+08,2023-12-19,2024-01-21,0.298481,4.424133e+07,2.981588e+08,-2.238160,-10.904971,-5.952512
3,dogwifhat,WIF,146,9.989202e+08,9.989202e+08,2023-12-19,2024-01-28,0.330760,5.211446e+07,3.304026e+08,3.782789,-4.029262,10.814297
4,dogwifhat,WIF,195,9.989202e+08,9.989202e+08,2023-12-19,2024-02-04,0.212275,3.530987e+07,2.120454e+08,-0.237957,-12.570497,-35.822098
5,dogwifhat,WIF,164,9.989202e+08,9.989202e+08,2023-12-19,2024-02-11,0.296083,3.277814e+07,2.957632e+08,-3.350781,-6.563455,39.481053
6,dogwifhat,WIF,145,9.989202e+08,9.989202e+08,2023-12-19,2024-02-18,0.409355,4.595516e+07,4.089134e+08,-1.200170,6.397954,38.256996
7,dogwifhat,WIF,166,9.989202e+08,9.989202e+08,2023-12-19,2024-02-25,0.349052,3.240654e+07,3.486754e+08,-0.899059,2.334497,-14.731219
8,dogwifhat,WIF,63,9.989202e+08,9.989202e+08,2023-12-19,2024-03-03,1.610116,4.179296e+08,1.608377e+09,-6.066608,16.010788,361.282145
9,dogwifhat,WIF,58,9.989202e+08,9.989202e+08,2023-12-19,2024-03-10,2.136817,3.565748e+08,2.134509e+09,4.599329,-2.127335,32.711972
